In [1]:
import pathlib
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import tarfile
from os import path
from tensorflow.keras import layers
from tensorflow import keras
import python_splitter
import shutil
from tensorflow.keras.models import load_model

import zipfile
from PIL import Image
from bs4 import BeautifulSoup

In [2]:
with zipfile.ZipFile('background_archive.zip', 'r') as zip_ref:
    for file in zip_ref.filelist:
        image = file.filename.split('/')
        if image[0] == 'images' and image[1] != '1000097.jpg':
            zip_ref.extract(file.filename, path='/data/bg-not_a_dog/')


In [3]:
if path.isdir('/data/dataset/Images/n99999998-Zzbackground'):
    shutil.rmtree("/data/dataset/Images/n99999998-Zzbackground")
    
os.mkdir('/data/dataset/Images/n99999998-Zzbackground/')
    
files = os.listdir('bg-not_a_dog/images')
i = 1
for file in files:
    shutil.move("/data/bg-not_a_dog/images/" + file, "/data/dataset/Images/n99999998-Zzbackground/n99999998_" + str(i).rjust(4, '0') + ".jpg")
    i += 1
    
shutil.rmtree("bg-not_a_dog")

print(len(os.listdir('/data/dataset/Images/n99999998-Zzbackground')))

714


In [4]:
with zipfile.ZipFile('human_archive.zip', 'r') as zip_ref:
    for file in zip_ref.filelist:
        image = file.filename.split('/')
        if image[0] == 'images':
            zip_ref.extract(file.filename, path='/data/bg-not_a_dog/')

In [5]:
if path.isdir('/data/dataset/Images/n99999999-Zzhumain'):
    shutil.rmtree("/data/dataset/Images/n99999999-Zzhumain")
    
os.mkdir('/data/dataset/Images/n99999999-Zzhumain/')
    
files = os.listdir('bg-not_a_dog/images/images')
i = 1
for file in files:
    shutil.move("/data/bg-not_a_dog/images/images/" + file, "/data/dataset/Images/n99999999-Zzhumain/n99999999_" + str(i).rjust(4, '0') + ".jpg")
    i += 1
    
shutil.rmtree("bg-not_a_dog")

print(len(os.listdir('/data/dataset/Images/n99999999-Zzhumain')))

2054


In [6]:
with zipfile.ZipFile('human_none_humain_archive.zip', 'r') as zip_ref:
    zip_ref.extractall('/data/bg-not_a_dog/')

In [7]:
files = os.listdir('bg-not_a_dog/human detection dataset/0')
for file in files:
    image_png = 'bg-not_a_dog/human detection dataset/0/' + file
    image_jpg = 'bg-not_a_dog/human detection dataset/0/' + file.replace('.png', '.jpg')
    image = Image.open(image_png)
    image = image.convert('RGB')
    image.save(image_jpg)
    os.remove(image_png)
    shutil.move(image_jpg, "/data/dataset/Images/n99999998-Zzbackground/n99999998_" + str(i).rjust(4, '0') + ".jpg")
    i += 1
    
print(len(os.listdir('/data/dataset/Images/n99999998-Zzbackground')))
      
# files = os.listdir('bg-not_a_dog/human detection dataset/1')
# for file in files:
#     image_png = 'bg-not_a_dog/human detection dataset/1/' + file
#     image_jpg = 'bg-not_a_dog/human detection dataset/1/' + file.replace('.png', '.jpg')
#     image = Image.open(image_png)
#     image = image.convert('RGB')
#     image.save(image_jpg)
#     os.remove(image_png)
#     shutil.move(image_jpg, "/data/dataset/Images/n99999999-Zzhuman/n99999999_" + str(i).rjust(4, '0') + ".jpg")
#     i += 1
    
# shutil.rmtree("bg-not_a_dog")
      
# print(len(os.listdir('/data/dataset/Images/n99999999-Zzhuman')))

1076


In [8]:
if not path.isdir('annotation'):
    data_dir = tf.keras.utils.get_file(
        "annotation.tar",
        "http://vision.stanford.edu/aditya86/ImageNetDogs/annotation.tar",
        extract=False)
    with tarfile.open(data_dir, 'r') as tar_ref:
        tar_ref.extractall(path='/data/annotation/')
    print('Dataset installé')
else:
    print('Dataset déja installé')

Dataset installé


In [9]:
if path.isdir('custom_dataset'):
    shutil.rmtree("custom_dataset")
os.mkdir('custom_dataset')
os.mkdir('custom_dataset/Images')

directories = os.listdir('annotation/Annotation')
for directory in directories:
    os.mkdir('custom_dataset/Images/' + directory)
    files = os.listdir('annotation/Annotation/' + directory)
    for file in files:
        old_name = 'annotation/Annotation/' + directory + '/' + file
        new_name = 'annotation/Annotation/' + directory + '/' + file + '.xml'
        os.rename(old_name, new_name)
        if path.exists("dataset/Images/" + directory + '/' + file + '.jpg'):
            with open(new_name, 'r') as f:
                data = f.read()
            Bs_data = BeautifulSoup(data, "xml")
            boxes = Bs_data.find('xmin')
            for i in range(len(boxes)):
                xmin = int(str(Bs_data.find_all('xmin')[i]).replace('<xmin>', '').replace('</xmin>', ''))
                ymin = int(str(Bs_data.find_all('ymin')[i]).replace('<ymin>', '').replace('</ymin>', ''))
                xmax = int(str(Bs_data.find_all('xmax')[i]).replace('<xmax>', '').replace('</xmax>', ''))
                ymax = int(str(Bs_data.find_all('ymax')[i]).replace('<ymax>', '').replace('</ymax>', ''))
                image = Image.open(r"dataset/Images/" + directory + "/" + file + ".jpg")
                image = image.convert('RGB')
                image = image.crop((xmin, ymin, xmax, ymax))
                image.save("custom_dataset/Images/" + directory + "/" + file + ".jpg")
                
os.rename(new_name, old_name)